In [33]:
import pandas as pd
import numpy as np
import math
import graphlab as gl
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import csv
%matplotlib inline

In [34]:
df=pd.read_csv('data/CAX_Bidding_TRAIN_Molecule_3_4_5.csv')
df['Start_date'] = pd.to_datetime(df['Start_date'])

In [35]:
def agg_competitor(data, index_start, index_end, skip):
    copy = data.copy()
    copy['Competitor'] = copy.iloc[:,1].map(lambda x: 0)
    i=index_start
    while i < index_end:
        copy.ix[:,i] = ~copy.ix[:,i].isnull()*1
        copy['Competitor'] = copy.iloc[:,i] + copy['Competitor']
        i=i+skip
    return copy['Competitor']

def max_price_per_presentation(data):
    return data['Winning_price_per_standard_unit'].max()

def days_since_loe(molecule_name, start_date):
    if molecule_name =='Molecule3':
        return (start_date - datetime(2009,1,31,0,0,0)).days
    else:
        return (start_date - datetime(2011,1,31,0,0,0)).days

def build_presentation_price_dict(data):
    p_presentation_dict  = {}
    for item in data['Presentation'].unique():
        row = data.ix[data[data['Presentation'] == item]['Winning_price_per_standard_unit'].idxmax()]
        if row['Tenure'] > 0:
            p_presentation_dict[row['Presentation']] = row['Winning_price_per_standard_unit']*row['Tenure']/30
        else:
            p_presentation_dict[row['Presentation']] = row['Winning_price_per_standard_unit']
    return p_presentation_dict

In [36]:
df['Competitor'] = agg_competitor(df, 16, 34, 2)
col=['Molecule','Winning_price_per_standard_unit','Province','Region','Tender_Type', 'Length_of_contract_in_Months', 'Presentation', 'Start_date', 'Competitor']
df2 = df[col]
df2.dropna()
df2['Tenure'] = df2.apply(lambda row: days_since_loe(row['Molecule'], row['Start_date']), axis=1)
p_presentation_dict_train = build_presentation_price_dict(df2)
max_presentation_train = max_price_per_presentation(df2)
df2['Presentation'] = df2.apply(lambda row: p_presentation_dict_train[row['Presentation']]/max_presentation_train, axis=1)

/Users/ssing25/anaconda2/envs/dato-env/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ssing25/anaconda2/envs/dato-env/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [37]:
df2['Length_of_contract_in_Months'] = df2.apply(lambda row: np.log(row['Length_of_contract_in_Months']), axis=1)
df2=df2[df2['Tenure'] >= 0]

/Users/ssing25/anaconda2/envs/dato-env/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [38]:
df2=df2.drop(['Molecule', 'Start_date', 'Region', 'Province', 'Tender_Type'],1)



In [39]:
df2.dropna()

,Winning_price_per_standard_unit,Length_of_contract_in_Months,Presentation,Competitor,Tenure
1,1.563530,2.995732,0.221927,5,339
2,1.770000,3.178054,0.221927,4,339
3,1.770000,3.178054,0.221927,5,341
4,1.500000,2.484907,0.221927,4,342
5,1.450000,3.583519,0.221927,4,345
6,1.812000,2.484907,0.221927,2,684
7,1.310000,3.178054,0.221927,6,786
8,1.294850,3.583519,0.221927,7,705
9,1.540000,3.583519,0.221927,5,1003
10,1.309860,3.583519,0.221927,6,710


In [16]:
df2=pd.get_dummies(df2)
y=df2.as_matrix()[:,0]

In [17]:
y.shape

(642,)

In [18]:
X=df2.as_matrix()[:,1:]

In [19]:
X.shape

(642, 4)

In [20]:
X_train,X_test,y_train,y_test=train_test_split(X,y)

In [21]:
reg=linear_model.LinearRegression()
reg.fit(X_train,y_train)
reg.score(X_test,y_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
reg.coef_

In [ ]:
y_pred=reg.predict(X_test)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_test, y_pred)
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

In [ ]:
y_test[0:10]

In [ ]:
y_pred[0:10]

In [ ]:
mean_squared_error(y_test,y_pred)